# Trigger files
This notebook is intended to create the trigger files for the eeg data analysis.

In [8]:
import copy
import datetime
import itertools
import os

import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display
from matplotlib.ticker import FormatStrFormatter

import pandas as pd
import pyxdf
import seaborn as sns
from scipy.signal import find_peaks
from tqdm.notebook import tqdm

In [19]:
files = os.listdir("data")  # get all files from the folder "data"
files.sort()  # sort them alphabetically
recordings = {}
for i, file in enumerate(files):  # store and display all files
    created = os.path.getmtime(f"data/{file}")  # creation timestamp
    created = datetime.datetime.fromtimestamp(created)  # translate as datetime
    created = created.strftime("%d.%m.%Y %H:%M")  # arrange it
    recordings[i] = {"file": file, "created": created}

files = [f.split(".")[0] for f in files]
print("Included:")
display(recordings)

Included:


{0: {'file': '39_room1_251022.xdf', 'created': '25.10.2022 16:40'},
 1: {'file': '40_room1_261022.xdf', 'created': '26.10.2022 16:54'}}

## 1. Load data

In [114]:
# check streams for recording 0
streams, _ = pyxdf.load_xdf(f"data/{recordings[0]['file']}")
display(streams[8]['time_series'])

array([[3.9710000e+03, 1.0000000e+00, 9.9704880e+01],
       [3.9720000e+03, 0.0000000e+00, 9.9716118e+01],
       [3.9730000e+03, 0.0000000e+00, 9.9727005e+01],
       ...,
       [3.0584200e+05, 9.9000000e+01, 3.8008079e+03],
       [3.0584300e+05, 9.9000000e+01, 3.8008191e+03],
       [3.0584400e+05, 9.9000000e+01, 3.8008301e+03]], dtype=float32)

In [156]:
pd.DataFrame(streams[7]['time_series'])[1]

0                                             fixationCross
1         img.1600x1000.date.2022-07-12_00-09-21.hitname...
2         img.1600x1000.date.2022-07-12_00-09-21.hitname...
3         img.1600x1000.date.2022-07-12_00-09-21.hitname...
4         img.1600x1000.date.2022-07-12_00-09-21.hitname...
                                ...                        
301983                                           endMessage
301984                                           endMessage
301985                                           endMessage
301986                                           endMessage
301987                                           endMessage
Name: 1, Length: 301988, dtype: object

In [175]:
type(streams[7]['time_series'])

list

In [176]:
s = streams[7]['time_series']
# s[:,1]

In [178]:

if isinstance(s, (list,pd.core.series.Series,np.ndarray)):
    stream_data = pd.DataFrame(streams[7]['time_series'])[1]
else:
    stream_data = s[:,1]
print(type(stream_data))

<class 'pandas.core.series.Series'>


In [ ]:
# stream channel names in recording 0
s_channels = {streams[i]["info"]["name"][0]: i for i in range(len(streams))}
s_channels

In [23]:
# useful functions
def correct_timestamps(ts):  # calculate time values from 0
    corrected = [0]
    length = len(ts)
    [corrected.append(ts[i + 1] - ts[0]) for i in range(length) if i < length - 1]
    return np.array(corrected)

def select_streams(streams):
    # stream names
    names_ch = "ImageInfo"
    # e_ch_name = "openvibeSignal"

    # get all current streams with their positions on the recording
    # example: {'Diode': 0, 'Audio': 1, 'openvibeSignal': 2}
    s_channels = {streams[i]["info"]["name"][0]: i for i in range(len(streams))}

    # store and return their positions
    u = s_channels[names_ch]
    # e = s_channels[e_ch_name]  # eeg stream channel (diode and microphone)
    return u


In [191]:
def get_streams_channel(streams, streams_keep=['ImageInfo','Visual']):
    data = pd.DataFrame()
    for i, ch_name in enumerate(streams_keep):
        # get all current streams with their positions on the recording
        # example: {'ImagesOrder': 0, 'ValidationError': 1, 'HeadTracking': 2}
        s_channels = {streams[i]["info"]["name"][0]: i for i in range(len(streams))}
        u = s_channels[ch_name]
        # get timestamps and append to df
        time_stamps =  streams[u]['time_stamps']
        print(type(time_stamps))
        # check the type and length of data arrays and get only 1 value of the array
        stream_data = streams[u]['time_series']
        if isinstance(stream_data, (list,pd.core.series.Series,np.ndarray)):
            stream_data = pd.DataFrame(streams[u]['time_series'])[1]
            print(type(stream_data))
            # data = pd.concat([data, pd.DataFrame(stream_data, columns=[f"data_{ch_name}"])], axis=1)
        else:
            stream_data = stream_data[:,1]
            print(type(stream_data))

        # data.append()
        data = pd.concat([data, pd.DataFrame(time_stamps, columns=[f"time_stamps_{ch_name}"])], axis=1)
        data[f"data_{ch_name}"] = stream_data
        # data.concat([time_stamps, columns=[f"{ch_name}"]),ignore_index=True)
        # data[f"{ch_name}"] = data.append()

    return data

In [192]:
df = get_streams_channel(streams, streams_keep=['ImageInfo','Visual'])
df

<class 'numpy.ndarray'>
<class 'pandas.core.series.Series'>
<class 'numpy.ndarray'>
<class 'pandas.core.series.Series'>


,time_stamps_ImageInfo,data_ImageInfo,time_stamps_Visual,data_Visual
0,934259.969858,fixationCross,934259.969854,1.0
1,934259.981103,img.1600x1000.date.2022-07-12_00-09-21.hitname...,934259.981097,0.0
2,934259.991990,img.1600x1000.date.2022-07-12_00-09-21.hitname...,934259.991984,0.0
3,934260.003191,img.1600x1000.date.2022-07-12_00-09-21.hitname...,934260.003185,0.0
4,934260.015160,img.1600x1000.date.2022-07-12_00-09-21.hitname...,934260.015151,0.0
...,...,...,...,...
301983,937961.059559,endMessage,937961.059551,99.0
301984,937961.070648,endMessage,937961.070642,99.0
301985,937961.081876,endMessage,937961.081869,99.0
301986,937961.092953,endMessage,937961.092945,99.0


In [ ]:
    for k in recordings[0]:
        file = recordings[k]["file"]
        recordings[k]['ImageInfo'] = {}

        #load file
        streams, file_header = pyxdf.load_xdf(f"data/{file}")



        # keys to keep from streams[ch]['footer']
